In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
p = .1
df_main = pd.read_csv('spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and random.random() > p)
df_main

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Bright Side,HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,The Len Price 3,If You Live Round Here,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Cocktail Slippers,In The City,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Little Lovin',HARD ROCK 2010
...,...,...,...,...
1288931,2302bf9c64dc63d88a750215ed187f2c,Pearl Jam,Jeremy,iPhone
1288932,2302bf9c64dc63d88a750215ed187f2c,Led Zeppelin,Rock & Roll,iPhone
1288933,2302bf9c64dc63d88a750215ed187f2c,Alice In Chains,Rooster,iPhone
1288934,2302bf9c64dc63d88a750215ed187f2c,Van Halen,Summer Nights,iPhone


In [3]:
df_main.columns = df_main.columns.str.replace('"', '')
df_main.columns = df_main.columns.str.replace('name', '')
df_main.columns = df_main.columns.str.replace(' ', '')
df_main.columns

Index(['user_id', 'artist', 'track', 'playlist'], dtype='object')

In [4]:
df_main = df_main.groupby('artist').filter(lambda x : len(x)>=50)

In [5]:
df_main = df_main[df_main.groupby('user_id').artist.transform('nunique') >= 10]

In [6]:
size = lambda x: len(x)
df_freq = df_main.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

,user_id,artist,freq
366936,defced0ece4ce946160b0d2698142eac,Vitamin String Quartet,327
65119,26b51e580277e131f87e4c7ee4c0887a,Vitamin String Quartet,324
108791,414050deadb38aafd8d4ad22ca634055,Vitamin String Quartet,263
2902,014e695cc6df96011b90a5beb3206012,Ilaiyaraaja,221
293072,b1d4116e7cf150ae7d77413620f5f571,Wolfgang Amadeus Mozart,219


In [7]:
df_artist = pd.DataFrame(df_freq["artist"].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist'})
df_artist.head()

,artist_id,artist
0,0,Vitamin String Quartet
1,1,Ilaiyaraaja
2,2,Wolfgang Amadeus Mozart
3,3,Grateful Dead
4,4,Lata Mangeshkar


In [8]:
df_freq  = pd.merge(df_freq , df_artist, how='inner', on='artist')
df_freq.head()

,user_id,artist,freq,artist_id
0,defced0ece4ce946160b0d2698142eac,Vitamin String Quartet,327,0
1,26b51e580277e131f87e4c7ee4c0887a,Vitamin String Quartet,324,0
2,414050deadb38aafd8d4ad22ca634055,Vitamin String Quartet,263,0
3,5a7c1187380dfd62b4667511333ba4c5,Vitamin String Quartet,37,0
4,e78e1e7b93c32bc27bf458f6cb8a5554,Vitamin String Quartet,33,0


In [9]:
df_pivot = df_freq.pivot(index='user_id',columns='artist_id',values='freq')
df_pivot

artist_id,0,1,2,3,4,5,6,7,8,9,...,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084
user_id,,,,,,,,,,,,,,,,,,,,,
00055176fea33f6e027cd3302289378b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000c11a16c89aa4b14b328080f5954ee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00123e0f544dee3ab006aa7f1e5725a7,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00139e9cb50fb309549e1561b476226d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0019363a0d57e94d39988c31eeb8d015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffc9e5f9869491983a67f254ab6e61a6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffe11226cdea81a2db9262c0ec7f5d71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff60baf392613ed33f745b89a9b38f7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_pivot = df_pivot.fillna(0)
df_pivot

artist_id,0,1,2,3,4,5,6,7,8,9,...,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084
user_id,,,,,,,,,,,,,,,,,,,,,
00055176fea33f6e027cd3302289378b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c11a16c89aa4b14b328080f5954ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00123e0f544dee3ab006aa7f1e5725a7,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00139e9cb50fb309549e1561b476226d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0019363a0d57e94d39988c31eeb8d015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffc9e5f9869491983a67f254ab6e61a6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffe11226cdea81a2db9262c0ec7f5d71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff60baf392613ed33f745b89a9b38f7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from sklearn.neighbors import NearestNeighbors
knn=NearestNeighbors(n_neighbors=50,metric='cosine')
Model=knn.fit(df_pivot)

In [12]:
user = df_pivot.iloc[1,]
user

artist_id
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
4080    0.0
4081    0.0
4082    0.0
4083    0.0
4084    0.0
Name: 000c11a16c89aa4b14b328080f5954ee, Length: 4085, dtype: float64

In [13]:
distances, indices = Model.kneighbors([user])
distances, indices

(array([[0.        , 0.68133786, 0.70299195, 0.7047892 , 0.7083237 ,
         0.71110959, 0.71600554, 0.72000117, 0.7339589 , 0.73485504,
         0.73589375, 0.73763003, 0.73786245, 0.73896075, 0.73934439,
         0.73939993, 0.74144388, 0.74704481, 0.75123371, 0.75809039,
         0.76174495, 0.76273263, 0.76475435, 0.76584727, 0.76943603,
         0.7718018 , 0.77274068, 0.77373231, 0.77537072, 0.77608383,
         0.77671846, 0.77748196, 0.77843517, 0.77861197, 0.77871431,
         0.77903712, 0.78241916, 0.78341306, 0.78392088, 0.78522523,
         0.78567054, 0.78635294, 0.78642276, 0.78755408, 0.78863671,
         0.79000087, 0.79405674, 0.79542813, 0.79594386, 0.79650007]]),
 array([[   1,  180, 5636, 7267, 8181, 1002, 2071, 6880, 3355, 7226, 4445,
         4738, 2180, 2851, 6874, 7820, 5473, 4783, 9035,  196, 2401, 8903,
          525, 2546, 7882, 4705, 2823, 7331, 4091, 2928, 8859, 6107, 9382,
         2510, 1913, 7505, 9567, 6600, 2126, 3181, 5551, 2900,  898, 2827,
       

In [14]:
neighbors=[]
for item in indices[0][1:]:
    try:
        neighbors.append(df_main[df_main.index == item]["user_id"].values[0])
    except IndexError:
        continue
neighbors = list(set(neighbors))
neighbors

['944c80d26922ae634d6ce445b1fdff7f',
 '123122ce28f91ca1cb208d73abde5405',
 'b86510df7a1b78877f8ea6734f503c29',
 '7429488f7c8927077b9fd9f03fee8f15',
 '53a80b4f5d73275d7cc00cc78ec812cd',
 'dd07906c75e6294caf540e61e2466429',
 '6cdb890a9fff57be31b072777350e632',
 'ee4c2080a04f5bba5176ce0bdd927f14',
 'db937456654d2465292c4daa947c95de',
 '89d9830c8028e7e719064b1203a128b9',
 'e7f64d23e67f83ce714ac01fd69380c6',
 '3efa79b30d043e3920c796b489696b1f',
 'ac146ad0c9870b5662519faf726a5178',
 '55b8a84507731cf8be9c13302ce6cbb9',
 '1ed9910b0db7fcb779ec65b2ded4892f',
 '76c64f65bd8b0b7627652af1ecceab03',
 'c50566d83fba17b20697039d5824db78',
 'f4c6fd1ac32cfc2a720aa1cc8af9d4e0',
 '58b1a0122f8b556e2a01b8138a557038',
 '6281783aff449ec0998865aa5bc782ec',
 '83503c92f7b93542f40c9ba2174a51a1',
 '6fd8cee0e5c7241a5f963e727781c23f']

In [15]:
df_neighbor=pd.DataFrame(columns=['user_id', 'artist','freq','artist_id'])
for item in neighbors:
    df_neighbor = df_neighbor.append(df_freq[df_freq.user_id == item],ignore_index = True)
df_neighbor

,user_id,artist,freq,artist_id
0,944c80d26922ae634d6ce445b1fdff7f,Glee Cast,1,69
1,944c80d26922ae634d6ce445b1fdff7f,Elton John,1,75
2,944c80d26922ae634d6ce445b1fdff7f,Belle & Sebastian,1,227
3,944c80d26922ae634d6ce445b1fdff7f,Frank Turner,8,314
4,944c80d26922ae634d6ce445b1fdff7f,Coldplay,1,366
...,...,...,...,...
1685,6fd8cee0e5c7241a5f963e727781c23f,The Raconteurs,1,3115
1686,6fd8cee0e5c7241a5f963e727781c23f,Chumbawamba,1,3574
1687,6fd8cee0e5c7241a5f963e727781c23f,Dick Dale,1,3747
1688,6fd8cee0e5c7241a5f963e727781c23f,Frankie Valli,1,3959


In [16]:
df_neighbor=df_neighbor.sort_values('freq',ascending=False)
df_neighbor

,user_id,artist,freq,artist_id
570,db937456654d2465292c4daa947c95de,Johnny Cash,108,14
586,db937456654d2465292c4daa947c95de,Red Hot Chili Peppers,35,167
579,db937456654d2465292c4daa947c95de,2Pac,33,79
370,6cdb890a9fff57be31b072777350e632,Blur,30,340
590,db937456654d2465292c4daa947c95de,Aerosmith,26,204
...,...,...,...,...
692,89d9830c8028e7e719064b1203a128b9,Kanye West,1,226
691,89d9830c8028e7e719064b1203a128b9,Radiohead,1,174
689,89d9830c8028e7e719064b1203a128b9,Johnny Cash,1,14
688,db937456654d2465292c4daa947c95de,Gigi D'Agostino,1,4000


In [17]:
#df_neighbor.groupby('Team').sum()
df_neighbor1 = pd.DataFrame({'artist':df_neighbor['artist'], 'freq':df_neighbor['freq']})
df_neighbor1 = df_neighbor1.groupby('artist').sum()

In [18]:
df_neighbor1.sort_values('freq',ascending=False, inplace = True)
df_neighbor1 = df_neighbor1.reset_index()

In [19]:
df_neighbor1[0:10]

,artist,freq
0,Johnny Cash,111
1,Red Hot Chili Peppers,44
2,JAY Z,38
3,2Pac,36
4,Blur,32
5,Lana Del Rey,29
6,Aerosmith,28
7,Beyoncé,22
8,AC/DC,22
9,Rihanna,21


# Testing

In [63]:
user_playlists = []
for artist in df_freq.loc[df_freq.user_id == '00055176fea33f6e027cd3302289378b']['artist']:
    user_playlists.extend(df_main.loc[df_main['artist'] == artist].playlist.squeeze().unique())
print(len(user_playlists))

2765


In [48]:
target_artists = []
for playlist in neighbor_playlists[:10]:
    target_artists.extend(df_main.loc[df_main['playlist'] == playlist].artist.squeeze())
print(len(target_artists))
print(target_artists[:10])
target_artists = list(set(target_artists))
print(len(target_artists))
print(target_artists[:10])

91097
['Darren Korb', 'Murray Gold', 'Michael Kiwanuka', 'Archive', 'Rise Against', 'Korn', 'Justice', 'C418', 'Michael Kiwanuka', 'Pendulum']
3987
['Angus & Julia Stone', 'Quantic', "The La's", 'Jack White', 'House Of Pain', 'MIKA', 'Thunder', 'The Neighbourhood', 'Marina and The Diamonds', 'Mike Posner']


In [61]:
size = lambda x: len(x)
df_playlist_count = df_main.groupby(['playlist']).agg('size').reset_index().rename(columns={0:'freq'})[['playlist', 'freq']].sort_values(['freq'], ascending=False)
df_playlist_count = df_playlist_count.reset_index(drop = True)


85187

In [87]:
top = df_playlist_count[0:len(df_playlist_count)//3].sample(n = 100)
#top

In [79]:
#top = df_playlist_count[df_playlist_count['playlist'].isin(user_playlists)][0:1]

,playlist,freq
0,Starred,90912


In [88]:
y_artist = df_main[df_main['playlist'].isin(top["playlist"])].artist.squeeze().unique()
print(len(y_artist))
len(list(set(y_artist) & set(df_neighbor1.artist[0:10])))

661


7

In [77]:
df_freq.loc[df_freq.user_id == '00055176fea33f6e027cd3302289378b']

,user_id,artist,freq,artist_id
102507,00055176fea33f6e027cd3302289378b,blink-182,1,585
115179,00055176fea33f6e027cd3302289378b,Simple Plan,1,669
154996,00055176fea33f6e027cd3302289378b,All Time Low,1,987
155466,00055176fea33f6e027cd3302289378b,One Direction,1,992
195236,00055176fea33f6e027cd3302289378b,5 Seconds Of Summer,1,1346
208447,00055176fea33f6e027cd3302289378b,Bruno Mars,1,1455
306894,00055176fea33f6e027cd3302289378b,OneRepublic,2,2486
321925,00055176fea33f6e027cd3302289378b,Natalie Imbruglia,1,2646
338051,00055176fea33f6e027cd3302289378b,Little Mix,1,2866
340302,00055176fea33f6e027cd3302289378b,Charli XCX,1,2897
